# Get the latest resource from a given dataset on data.public.lu

Let's suppose that you need to download the latest resource from a dataset every month. This file will then be imported into your database. With the API available on data.public.lu, you can automate this download. 

Let's initialise some stuff:

In [29]:
import requests, re, os
from dotenv import load_dotenv 
load_dotenv()
API = os.environ.get('API')
API_KEY = os.environ.get('API_KEY')
headers = {'X-API-KEY': API_KEY}

## Parameters

Here we need to set the identifier of the dataset. You can for example find it in the url of a given dataset.
In a dataset, the data producers can put different kind of files. Some of them even put the documentation as a resource. In this case, it can be helpful to filter the resources based on a regular expression. 

In [30]:
# the id of the Dataset you want to get
dataset = 'parc-automobile-du-luxembourg'

# the pattern matching the names of the files that can be downloaded
fPattern = '^Parc_Automobile_\\d{6}.xlsx$'

## Find the right resource

Here we call the API to get all the info about a dataset. This info contains the list of resources of this dataset. We filter these resources to keep only the ones matching the previously defined pattern and we sort them by publication date. 
The first resource of this list is thus the latest one and we can proceed to download it.

In [31]:
r = requests.get('{}/datasets/{}/'.format(API, dataset), headers=headers)
r.raise_for_status()

resources = r.json()['resources']
resources = list(filter(lambda x: re.search(fPattern, x['title']), resources))
resources.sort(key=lambda x: x['published'], reverse=True)

## Download the file

When the resource has been found, we have its URL on data.public.lu. We download the file and store it in the current folder.

In [32]:
if resources:
    print('Downloading: '+ resources[0]['title'])
    s = requests.get(resources[0]['url'], headers=headers)
    s.raise_for_status()
    with open(resources[0]['title'], 'wb') as f:
        f.write(s.content)
    print('Downloaded!')



Downloading: Parc_Automobile_202207.xlsx
Downloaded!
